In [ ]:
#| default_exp helpers.env

# Environment Detection helper (env.py)

> This helper centralizes **environment-specific setup** for notebooks that rely on external APIs (e.g. OpenAI, Anthropic).

The goal is to ensure that **the same notebook runs unchanged** in multiple contexts:

* Google Colab
* Hosted notebook platforms (e.g. Deepnote / Solveit)
* Local development environments

Rather than scattering environment checks throughout notebooks, all logic for
*detecting the execution environment* and *loading API keys* lives here.


In [ ]:
#| export
import os

In [ ]:
#| export
def load_env():
    """
    Load API keys in a platform-aware way.

    This function attempts to populate environment variables required by
    downstream LLM helpers. The strategy depends on where the notebook
    is running:

    - **Google Colab**: read keys from `google.colab.userdata`
    - **Local development**: load keys from a `.env` file if present
    - **Hosted environments** (e.g. Deepnote / Solveit): assume keys are already injected

    The function does not validate keys or make API calls.

    Returns
    -------
    str
        One of {"colab", "dotenv", "external"}, indicating which environment
        was detected.
    """
    try:
        import google.colab
        from google.colab import userdata

        os.environ.setdefault("ANTHROPIC_API_KEY", userdata.get("ANTHROPIC_API_KEY"))
        os.environ.setdefault("OPENAI_API_KEY", userdata.get("OPENAI_API_KEY"))
        return "colab"

    except ImportError:
        pass

    # Local dev
    if os.path.exists(".env"):
        from dotenv import load_dotenv
        load_dotenv()
        return "dotenv"

    # Deepnote / other hosted env
    return "external"


In [ ]:
# This cell is for interactive testing only.
# It is NOT exported as part of the library.
env = load_env()
print(f"Detected environment: {env}")

assert env in {"colab", "dotenv", "external"}

Detected environment: external
